Inspect the data inside the truthtriplet files.

These are the upstream data files we will make the prong embed training data from.

To make the file that goes into this notebook run:

Script found in `ubdl/larflow/larflow/KeyPoints/test`
```
python run_prepalldata.py -tb -tri -adc wiremc --input-larlite [has mcinfo] --input-larcv [has wire/truth images] -o test.root
```

In [ ]:
import ROOT as rt
import numpy as np
from larcv import larcv
from larflow import larflow
import chart_studio as cs
import chart_studio.plotly as py
import plotly.graph_objects as go
from ctypes import c_int
larcv.load_pyutil()

In [ ]:
# data useful for plots

import lardly
from lardly import DetectorOutline # load utility to draw TPC outline
detdata = DetectorOutline()
detlines = detdata.getlines(color=(10,10,10))

# PARTICLE LABEL COLORS
# from larcv/core/DataFormat/DataFormatTypes.h
#     kROIUnknown=0, ///< LArbys
#     kROICosmic,    ///< Cosmics
#     kROIBNB,       ///< BNB
#     kROIEminus,    ///< Electron
#     kROIGamma,     ///< Gamma
#     kROIPizero,    ///< Pi0
#     kROIMuminus,   ///< Muon
#     kROIKminus,    ///< Kaon
#     kROIPiminus,   ///< Charged Pion
#     kROIProton,    ///< Proton
#     kROITypeMax    ///< enum element counter
ssnetcolor = {0:np.array((0,0,0)),     # kROIUnknown                                                                                                                                                   
              1:np.array((255,0,0)),   # kROICosmic (not used)                                                                                                                                       
              2:np.array((0,255,0)),   # kROIBNB (not used)                                                                                                                             
              3:np.array((0,0,255)),   # kROIEminus (e-/e+)                                                                                                                                              
              4:np.array((255,0,255)), # kROIGamma                                                                                                                                                 
              5:np.array((0,255,255)), # kROIPizero                                                                                                                                            
              6:np.array((255,255,0)), # kROImuminus (mu-/mu+)
              7:np.array((123,300,10)),# kROIKminus (k+/k-)
              8:np.array((204,204,255)), # kROIPiminus (pi+/pi-)
              9:np.array((255, 165, 0))} # kProton

kpcolors = {0:np.array((255,0,0)), # nu
            1:np.array((0,255,0)), # track-start
            2:np.array((0,0,255)), # track-end
            3:np.array((255,255,0)), # shower
            4:np.array((0,255,255)), # michel
            5:np.array((255,0,255))} # delta


In [ ]:
#rootfiles = ["larmatchtriplet_bnbnu_0003.root"]
rootfiles = ["test.root"]
rootfile_v = rt.std.vector("string")()
for rf in rootfiles:
    rootfile_v.push_back( rf )

In [ ]:
loader = larflow.keypoints.LoaderKeypointData(rootfile_v)
loader.set_verbosity( larcv.msg.kDEBUG )

In [ ]:
ENTRY=7
nbytes = loader.load_entry(ENTRY)
print("number of bytes read: ",nbytes)

In [ ]:
# we get a dictionary of data in numpy array form

loader.triplet_v.at(0).setShuffleWhenSampling(False)
tripdata = loader.triplet_v.at(0).get_all_triplet_data( True )
spacepoints = loader.triplet_v.at(0).make_spacepoint_charge_array()
nfilled = c_int(0)
ntriplets = tripdata.shape[0]

data = loader.triplet_v.at(0).make_triplet_ndarray(False)

kpdata = loader.sample_data( ntriplets, nfilled, True )
data.update(kpdata)
data.update(spacepoints)
 
#print(data.keys())
print("LIST OF KEYS")
for k in data.keys():
    print(k,": ",data[k].shape)

In [ ]:
# PLOT PARTICLE CLUSTER LABELS

TRUE_ONLY=False
NU_ONLY=True
SHOW_ME_INDICES = []

if TRUE_ONLY:
    xyz = data['spacepoint_t'][ data['truetriplet_t']==1 ]
    trackid = data['instance_t'][ data['truetriplet_t']==1 ]
    origin  = data['origin_t'][ data['truetriplet_t']==1 ]
else:
    xyz     = data['spacepoint_t']
    trackid = data['instance_t']
    origin  = data['origin_t']
    
if NU_ONLY:
    xyz = xyz[origin==1]
    trackid = trackid[origin==1]
    
if xyz.shape[0]>200000:
    reduction_factor = 150e3/float(xyz.shape[0])
    keep = np.random.rand( xyz.shape[0] ) < reduction_factor
    xyz     = data['spacepoint_t'][keep==1]
    trackid = data['instance_t'][keep==1]
    origin  = data['origin_t'][keep==1]
    
print("xyz: ",xyz.shape)
print("trackids: ",trackid.shape)
print("origin: ",origin.shape)
print("origin ids: ",np.unique(origin))


uniqueids = np.unique(trackid)
print("number of unique ids: ",uniqueids.shape)

# PLOT LOOP
cluster_plots = []
for n,uid in enumerate(uniqueids):
    
    cluster_xyz = xyz[ trackid==uid ]
    #print("cluster for track[%d] has %d points"%(uid,cluster_xyz.shape[0]))
    randcol = np.random.randint(low=20,high=254,size=3)
    cluster_color = "rgb(%d,%d,%d)"%(randcol[0],randcol[1],randcol[2])
    c_plot = {
        "type":"scatter3d",
        "x": cluster_xyz[:,0],
        "y": cluster_xyz[:,1],
        "z": cluster_xyz[:,2],
        "mode":"markers",
        "name":"%d"%(uid),
        "marker":{"color":cluster_color,"size":2,"opacity":1.0},
    }
    
    if len(SHOW_ME_INDICES)>0:
        if uid not in SHOW_ME_INDICES:
            c_plot["marker"]["color"] = "rgb(25,25,25)"
            c_plot["marker"]["opacity"] = 0.2
            c_plot["marker"]["size"] = 1
        else:
            c_plot["marker"]["size"] = 2
    if uid==0:
        c_plot["marker"]["color"] = "rgb(170,170,170)"
        c_plot["marker"]["size"] = 1
        c_plot["marker"]["opacity"] = 0.1
    cluster_plots.append( c_plot )

detlines = detdata.getlines(color=(10,10,10))
plot_traces = detlines + cluster_plots

#PLOT VTX
if loader.has_larbysmc:
    vtx = np.zeros((1,3))
    vtx[0,0] = loader.vtx_sce_x
    vtx[0,1] = loader.vtx_sce_y
    vtx[0,2] = loader.vtx_sce_z
    print("VTX: ",vtx)
    plot_vtx = {
        "type":"scatter3d",
            "x": vtx[:,0],
            "y": vtx[:,1],
            "z": vtx[:,2],
            "mode":"markers",
            "name":"VTX"%(uid),
            "marker":{"color":"rgb(120,120,120)","size":10,"opacity":0.5},
        }
    plot_traces += [plot_vtx]

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='Particle Instance Labels',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_traces, layout=layout)
fig.show()


In [ ]:
# Plot SSNET (SEGMENT_T)

TRUE_ONLY=False
NU_ONLY=False
SHOW_ME_INDICES = []

if TRUE_ONLY:
    xyz = data['spacepoint_t'][ data['truetriplet_t']==1 ]
    ssnet = data['segment_t'][ data['truetriplet_t']==1 ]
    origin  = data['origin_t'][ data['truetriplet_t']==1 ]
else:
    xyz     = data['spacepoint_t']
    ssnet = data['segment_t']
    origin  = data['origin_t']
    
if NU_ONLY:
    xyz = xyz[origin==1]
    ssnet = ssnet[origin==1]

print("xyz: ",xyz.shape)
print("ssnet: ",ssnet.shape)
print("origin: ",origin.shape)
    
if xyz.shape[0]>200000:
    reduction_factor = 150e3/float(xyz.shape[0])
    keep = np.random.rand( xyz.shape[0] ) < reduction_factor
    xyz     = xyz[keep==1]
    ssnet = ssnet[keep==1]
    origin  = origin[keep==1]
    
print("xyz: ",xyz.shape)
print("ssnet: ",ssnet.shape)
print("origin: ",origin.shape)

uniqueids = np.unique(ssnet)
print("number of unique ids: ",uniqueids.shape)
print(uniqueids)

# SSNET
color_array = np.zeros( (xyz.shape[0],3) )
for n,pid in enumerate(uniqueids):
    
    color_array[ ssnet==pid ] = ssnetcolor[pid]
print("color array: ",color_array.shape)

ssnet_plot = {
    "type":"scatter3d",
    "x": xyz[:,0],
    "y": xyz[:,1],
    "z": xyz[:,2],
    "mode":"markers",
    "name":"ssnet labels",
    "marker":{"color":color_array,"size":2,"opacity":1.0},
}
    

detlines = detdata.getlines(color=(10,10,10))
ssnet_plot_traces = detlines + [ssnet_plot]

#PLOT VTX
if loader.has_larbysmc:
    vtx = np.zeros((1,3))
    vtx[0,0] = loader.vtx_sce_x
    vtx[0,1] = loader.vtx_sce_y
    vtx[0,2] = loader.vtx_sce_z
    print("VTX: ",vtx)
    plot_vtx = {
        "type":"scatter3d",
            "x": vtx[:,0],
            "y": vtx[:,1],
            "z": vtx[:,2],
            "mode":"markers",
            "name":"VTX",
            "marker":{"color":"rgb(120,120,120)","size":10,"opacity":0.5},
        }
    ssnet_plot_traces += [plot_vtx]

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='SSNET LABELS',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=ssnet_plot_traces, layout=layout)
fig.show()


In [ ]:
# Plot SSNET (SSNET_LABELS)

TRUE_ONLY=True
NU_ONLY=False
SHOW_ME_INDICES = []

if TRUE_ONLY:
    xyz = data['spacepoint_t'][ data['truetriplet_t']==1 ]
    ssnet = data['ssnet_label'][ data['truetriplet_t']==1 ]
    origin  = data['origin_t'][ data['truetriplet_t']==1 ]
else:
    xyz     = data['spacepoint_t']
    ssnet = data['ssnet_label']
    origin  = data['origin_t']

print("xyz: ",xyz.shape)
print("ssnet: ",ssnet.shape)
print("origin: ",origin.shape)

if NU_ONLY:
    xyz = xyz[origin==1]
    ssnet = ssnet[origin==1]
    
if xyz.shape[0]>200000:
    reduction_factor = 150e3/float(xyz.shape[0])
    keep = np.random.rand( xyz.shape[0] ) < reduction_factor
    xyz     = xyz[keep==1]
    ssnet   = ssnet[keep==1]
    
print("xyz: ",xyz.shape)
print("ssnet: ",ssnet.shape)

uniqueids = np.unique(ssnet)
print("number of unique ids: ",uniqueids.shape)
print(uniqueids)

# SSNET
color_array = np.zeros( (xyz.shape[0],3) )
for n,pid in enumerate(uniqueids):
    
    color_array[ ssnet==pid ] = ssnetcolor[pid]
print("color array: ",color_array.shape)

ssnet_plot = {
    "type":"scatter3d",
    "x": xyz[:,0],
    "y": xyz[:,1],
    "z": xyz[:,2],
    "mode":"markers",
    "name":"ssnet labels",
    "marker":{"color":color_array,"size":2,"opacity":1.0},
}
    

detlines = detdata.getlines(color=(10,10,10))
ssnet_plot_traces = detlines + [ssnet_plot]

#PLOT VTX
if loader.has_larbysmc:
    vtx = np.zeros((1,3))
    vtx[0,0] = loader.vtx_sce_x
    vtx[0,1] = loader.vtx_sce_y
    vtx[0,2] = loader.vtx_sce_z
    print("VTX: ",vtx)
    plot_vtx = {
        "type":"scatter3d",
            "x": vtx[:,0],
            "y": vtx[:,1],
            "z": vtx[:,2],
            "mode":"markers",
            "name":"VTX",
            "marker":{"color":"rgb(120,120,120)","size":10,"opacity":0.5},
        }
    ssnet_plot_traces += [plot_vtx]

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='SSNET LABELS',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=ssnet_plot_traces, layout=layout)
fig.show()


In [ ]:
# Plot KEYPOINT LABELS

KPTYPE=5
TRUE_ONLY=True
NU_ONLY=False
SHOW_ME_INDICES = []

if TRUE_ONLY:
    xyz = data['spacepoint_t'][ data['truetriplet_t']==1 ]
    kplabel = data['kplabel'][ data['truetriplet_t']==1 ]
    origin  = data['origin_t'][ data['truetriplet_t']==1 ]
else:
    xyz     = data['spacepoint_t']
    kplabel = data['kplabel']
    origin  = data['origin_t']

print("xyz: ",xyz.shape)
print("kplabel: ",kplabel.shape)
print("origin: ",origin.shape)

if NU_ONLY:
    xyz = xyz[origin==1]
    kplabel = kplabel[origin==1]
    
if xyz.shape[0]>200000:
    reduction_factor = 150e3/float(xyz.shape[0])
    keep = np.random.rand( xyz.shape[0] ) < reduction_factor
    xyz     = xyz[keep==1]
    kplabel   = kplabel[keep==1]
    
print("xyz: ",xyz.shape)
print("kplabel: ",kplabel.shape)

# KP Plots
kp_plots = []
for ikp in range(6):
    kpscore = kplabel[:,ikp]
    kptype_plot = {
        "type":"scatter3d",
        "x": xyz[:,0],
        "y": xyz[:,1],
        "z": xyz[:,2],
        "mode":"markers",
        "name":"kpscore[%d]"%(ikp),
        "marker":{"color":kpscore,"size":2,"opacity":1.0,'colorscale':'Viridis'},
    }
    kp_plots.append(kptype_plot)

detlines = detdata.getlines(color=(10,10,10))
keypoint_plot_traces = detlines + [kp_plots[KPTYPE]]

#PLOT VTX
if loader.has_larbysmc:
    vtx = np.zeros((1,3))
    vtx[0,0] = loader.vtx_sce_x
    vtx[0,1] = loader.vtx_sce_y
    vtx[0,2] = loader.vtx_sce_z
    print("VTX: ",vtx)
    plot_vtx = {
        "type":"scatter3d",
            "x": vtx[:,0],
            "y": vtx[:,1],
            "z": vtx[:,2],
            "mode":"markers",
            "name":"VTX",
            "marker":{"color":"rgb(120,120,120)","size":10,"opacity":0.5},
        }
    keypoint_plot_traces += [plot_vtx]

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='KEYPOINT TYPE=%d LABELS'%(KPTYPE),
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=keypoint_plot_traces, layout=layout)
fig.show()


In [ ]:
# Plot KEYPOINT MARKERS

# KP Plots
kp_marker_data_pos  = loader.get_keypoint_pos()
kp_marker_data_type = loader.get_keypoint_types()
numkp = kp_marker_data_pos.size()

kp_indices = {}
for ikp in range(numkp):
    ikptype = kp_marker_data_type.at(ikp)
    if ikptype not in kp_indices:
        kp_indices[ikptype] = []
    kp_indices[ikptype].append(ikp)

kpmarker_plots = []
for ikptype in range(6):
    if ikptype not in kp_indices:
        continue
    nkp = len(kp_indices[ikptype])
    kpxyz = np.zeros( (nkp,3) )
    for i,idx in enumerate(kp_indices[ikptype]):
        for v in range(3):
            kpxyz[i,v] = kp_marker_data_pos.at(idx)[v]
    
    zcolor = "rgb(%d,%d,%d)"%(kpcolors[ikptype][0],kpcolors[ikptype][1],kpcolors[ikptype][2])
    kptype_marker_plot = {
        "type":"scatter3d",
        "x": kpxyz[:,0],
        "y": kpxyz[:,1],
        "z": kpxyz[:,2],
        "mode":"markers",
        "name":"[%d]"%(ikptype),
        "marker":{"color":zcolor,"size":4,"opacity":1.0},
    }
    kpmarker_plots.append(kptype_marker_plot)

KPTYPE=0
TRUE_ONLY=True
NU_ONLY=False

if TRUE_ONLY:
    xyz = data['spacepoint_t'][ data['truetriplet_t']==1 ]
    kplabel = data['kplabel'][ data['truetriplet_t']==1 ]
    origin  = data['origin_t'][ data['truetriplet_t']==1 ]
else:
    xyz     = data['spacepoint_t']
    kplabel = data['kplabel']
    origin  = data['origin_t']

if NU_ONLY:
    xyz = xyz[origin==1]
    kplabel = kplabel[origin==1]
    
if xyz.shape[0]>200000:
    reduction_factor = 150e3/float(xyz.shape[0])
    keep = np.random.rand( xyz.shape[0] ) < reduction_factor
    xyz     = xyz[keep==1]
    kplabel   = kplabel[keep==1]

kpscore = kplabel[:,KPTYPE]
kptype_plot = {
    "type":"scatter3d",
    "x": xyz[:,0],
    "y": xyz[:,1],
    "z": xyz[:,2],
    "mode":"markers",
    "name":"kpscore[%d]"%(KPTYPE),
    "marker":{"color":kpscore,"size":2,"opacity":1.0,'colorscale':'Viridis'},
}

    
detlines = detdata.getlines(color=(10,10,10))
keypoint_marker_plot_traces = detlines + [kptype_plot] + kpmarker_plots

#PLOT VTX
if loader.has_larbysmc:
    vtx = np.zeros((1,3))
    vtx[0,0] = loader.vtx_sce_x
    vtx[0,1] = loader.vtx_sce_y
    vtx[0,2] = loader.vtx_sce_z
    print("VTX: ",vtx)
    plot_vtx = {
        "type":"scatter3d",
            "x": vtx[:,0],
            "y": vtx[:,1],
            "z": vtx[:,2],
            "mode":"markers",
            "name":"VTX",
            "marker":{"color":"rgb(120,120,120)","size":10,"opacity":0.5},
        }
    keypoint_marker_plot_traces += [plot_vtx]

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='KEYPOINT MARKERS',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=keypoint_marker_plot_traces, layout=layout)
fig.show()


In [ ]:
# Look at spacepoints by match span
# This is the difference in num wires between true intersection and spacepoint intersection on plane

NU_ONLY=True
MATCH_SPAN=2
# The default MATCH_SPAN used to set truetriplet_t tag is span<3

span    = data["truespan_t"]
xyz     = data['spacepoint_t']
origin  = data['origin_t']
truth   = data['truetriplet_t']
trackid = data['instance_t']
print("xyz: ",xyz.shape)
print("span: ",span.shape)

if NU_ONLY:
    xyz = xyz[origin==1]
    span = span[origin==1]
    truth = truth[origin==1]
    trackid = trackid[origin==1]
    print("post nu-only")
    print("xyz: ",xyz.shape)
    print("span: ",span.shape)
    print("trackid: ",trackid.shape)
    
if xyz.shape[0]>200000:
    reduction_factor = 150e3/float(xyz.shape[0])
    keep = np.random.rand( xyz.shape[0] ) < reduction_factor
    xyz   = xyz[keep==1]
    span  = span[keep==1]
    truth = truth[keep==1]
    trackid = trackid[keep==1]
    
print("xyz: ",xyz.shape)
print("span: ",span.shape)
print("truth: ",truth.shape)
print("instance: ",trackid.shape)
spantag = span<MATCH_SPAN

# PLOT LOOP
span_plots = []
for tag in [0,1,2]:
    
    if tag in [0,1]:
        cluster_xyz = xyz[ spantag==tag ]
        if tag==0:
            cluster_truth = truth[ spantag==0 ]
            cluster_xyz = cluster_xyz[ cluster_truth==0 ]
    else:
        cluster_xyz = xyz[ spantag==0 ]
        cluster_truth = truth[ spantag==0 ]
        cluster_xyz = cluster_xyz[ cluster_truth==1 ]
        
    print("TAG[",tag,"]: ",cluster_xyz.shape)
    if tag==0:
        cluster_color = "rgb(120,120,120)"
    elif tag==1:
        cluster_color = "rgb(200,0,0)"
    elif tag==2:
        cluster_color = "rgb(0,0,100)"
        
    c_plot = {
        "type":"scatter3d",
        "x": cluster_xyz[:,0],
        "y": cluster_xyz[:,1],
        "z": cluster_xyz[:,2],
        "mode":"markers",
        "name":"span<%d"%(MATCH_SPAN),
        "marker":{"color":cluster_color,"size":1,"opacity":1.0},
    }
    
    if tag in [0,2]:
        c_plot["marker"]["size"] = 1
        c_plot["marker"]["opacity"] = 0.1
    if tag in [1]:
        span_plots.append( c_plot )

detlines = detdata.getlines(color=(10,10,10))
span_plot_traces = detlines + span_plots

#PLOT VTX
if loader.has_larbysmc:
    vtx = np.zeros((1,3))
    vtx[0,0] = loader.vtx_sce_x
    vtx[0,1] = loader.vtx_sce_y
    vtx[0,2] = loader.vtx_sce_z
    print("VTX: ",vtx)
    plot_vtx = {
        "type":"scatter3d",
            "x": vtx[:,0],
            "y": vtx[:,1],
            "z": vtx[:,2],
            "mode":"markers",
            "name":"VTX",
            "marker":{"color":"rgb(120,120,120)","size":10,"opacity":0.5},
        }
    span_plot_traces += [plot_vtx]

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='Spacepoints with True labels when MATCH_SPAN=%d'%(MATCH_SPAN),
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=span_plot_traces, layout=layout)
fig.show()